In [1]:
from ensemble_boxes import *
import pandas as pd
import numpy as np

In [27]:
predictions = []
predictions.append(pd.read_csv("./work_dirs/2023_05_11_20_36_05_FasterRCNN_SGKF_0/submission_latest.csv"))
predictions.append(pd.read_csv("./work_dirs/2023_05_12_01_40_01_FasterRCNN_SGKF_4/submission_latest.csv"))
#predictions.append(pd.read_csv("./work_dirs/2023_05_11_23_27_17_FasterRCNN_SGKF_2/submission_latest.csv"))

prediction_strings = []
file_names = []

for img_idx in range(len(predictions[0])):

    boxes_list = []
    scores_list = []
    labels_list = []

    for model_idx in range(len(predictions)): 
        pred_str = predictions[model_idx].loc[img_idx]['PredictionString']
        if str(type(pred_str)) != "<class 'str'>":
            break
        else:
            pred = np.array(pred_str.strip().split(" ")).reshape(-1,6)

            box = pred[:, 2:].astype(np.float64)/1024  # 0~1 normalize
            score = pred[:, 1].astype(np.float64).flatten()
            label = pred[:, 0].astype(np.int64).flatten()

        boxes_list.append(box)
        scores_list.append(score)
        labels_list.append(label)
    
    
    boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=0.55)
    #category로 정렬
    sorted_data = sorted(zip(labels.tolist(), boxes.tolist(), scores.tolist()))
    sorted_labels, sorted_boxes, sorted_scores = zip(*sorted_data)

    prediction_string = ""
    for i in range(len(boxes)):
        prediction_string += str(int(sorted_labels[i])) + ' ' + str(sorted_scores[i]) + ' ' + str(sorted_boxes[i][0]*1024) + ' ' + str(sorted_boxes[i][1]*1024) + ' ' + str(sorted_boxes[i][2]*1024) + ' ' + str(sorted_boxes[i][3]*1024) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(predictions[model_idx].loc[img_idx]["image_id"])

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv("./work_dirs/submission/ensemble_nms(0,4).csv", index=None)
submission.head()

,PredictionString,image_id
0,0 0.07436533 24.516418 310.31064 228.86052 438...,test/0000.jpg
1,0 0.051838454 135.54947 3.022766 310.5842 270....,test/0001.jpg
2,0 0.07860429 0.0 225.59164 163.30222 254.09439...,test/0002.jpg
3,0 0.05273235 0.0 24.617798 522.2323 1017.9741 ...,test/0003.jpg
4,0 0.57932156 130.69745 370.28705 906.7723 765....,test/0004.jpg
